In [ ]:
#pip install cvrplib

In [2]:
from random import randrange, random, seed
from math import exp
import glob, os
import cvrplib
import sys
from datetime import datetime

MAX_COUNT = 10000000

# Считаем суммарную Capacity для маршрута
def calc_cap(route, cd):
    sum = 0
    for i in range(1,len(route)):
        sum = sum + cd[route[i]]
    return sum

# Считаем суммарную длину маршрута
def total_length(cr,var):
    sum = 0
    for r in range(len(cr)): # r - маршрут;
        for i in range(len(cr[r])): # i - клиенты;
            if cr[r][i+1] == 0:
                break
            # Сумма евклидовых расстояний всех маршрутов
            sum = sum + var.distances[cr[r][i]][cr[r][i+1]]
    return sum

# Смотрим индекс последнего клиента на маршруте
def last_client(route):
    i = 1
    while route[i] != 0:
        i += 1
    return i

def show_routes(cr,goal,result):
    for r in range(len(cr)): # Вывод результата
        print ("Route #",r+1, sep='', end=': ') 
        for i in range(1,len(cr[r])):
            if cr[r][i] != 0:
                print(cr[r][i], end=' ')
        print()
    if len2 > goal:
        print( "FAILED: ", )
    print( "Cost", len2 )

# Меняем два элемента (клиентов) местами
def cswap(cr,var,T,change):
    # Текущая длина маршрутов
    len1 = total_length( cr, var )
    # Создаём новый массив
    cr2 = [0] * len(cr)
    for i in range(len(cr)):
        cr2[i] = [0] * var.dimension
    # Поэлементно копируем весь маршрут на случай необходимости отката
    for i in range(len(cr)):
        for j in range(var.dimension):
            cr2[i][j] = cr[i][j]
    for k in range(change):
        r1 = randrange(len(cr))
        r2 = randrange(len(cr))
        # Последние колонки в маршрутах
        lc1 = last_client(cr[r1])+1
        lc2 = last_client(cr[r2])+1
        # Выбираем рандомных клиентов из маршрутов (может быть и в рамках одного маршрута)
        i1 = randrange(1,lc1)
        i2 = randrange(1,lc2)

        if( (cr[r1][i1] != 0) & (cr[r2][i2] != 0) ): # Меняем клиента на клиента, если в маршруте есть хотя бы по 1 клиенту
            if cr[r1][i1] != cr[r2][i2]:
                swap = cr[r1][i1]
                cr[r1][i1] = cr[r2][i2]
                cr[r2][i2] = swap
                #len2 = total_length( cr, var )
        else: # Перебрасываем клиента с маршрута на маршрут
            if ((cr[r1][i1] + cr[r2][i2]) != 0) & (r1 != r2):
                if( r1 != r2 ): # Действие имеет смысл только между разными маршрутами
                    # Ищем пустой маршрут среди двух
                    if cr[r1][i1] != 0:
                        # Если первый элемент не пустой, меняем ячейки местами
                        k = r1
                        r1 = r2
                        r2 = k
                        k = i1
                        i1 = i2
                        i2 = k
                        k = lc1
                        lc1 = lc2
                        lc2 = k
                    k = cr[r1][i1]
                    cr[r1][i1] = cr[r2][i2]
                    cr[r2][i2] = 0
                    i1 = randrange(1,lc1) # Выбираем позицию в маршруте для нового клиента
                    while k != 0: # Вставляем клиента в маршрут и сдвигаем существующих клиентов вправо
                        i1 += 1
                        l = cr[r1][i1]
                        cr[r1][i1] = k
                        k = l
                    i = i2 + 1
                    while cr[r2][i] != 0: # Перемещаем клиентов влево
                        cr[r2][i-1] = cr[r2][i]
                        cr[r2][i] = 0
                        i += 1
    ret = 1 # Изначально считаем, что перестановка удалась
    for r in range(len(cr)): 
        if calc_cap(cr[r],var.demands) > var.capacity: # Проверяем, не превысили ли мы вместимость 
            ret = 0
            break
    if ret == 1: 
        len2 = total_length(cr,var)
        if (len2 > len1): # Если длина стала больше, по экспоненте определяем вероятность перестановки
            p = exp((len1-len2)/T)
            q = random()
            if p <= q:
                ret = 0 # Отмена перестановки
    if ret == 0:
        for i in range(len(cr)):
            for j in range(var.dimension):
                cr[i][j] = cr2[i][j]
    return ret # 0 - неудачный обмен, 1 - удачный обмен

entries = glob.glob('datafiles\\*.vrp')
for file in entries:
    var = cvrplib.read(file)
    trucks = int(var.name[var.name.find('-k')+2:])
    fr = open(file)
    skip = fr.readline()    # NAME
    val = fr.readline()     # COMMENT
    data = val.split(" ")
    goal = int(data[len(data)-1][:-2])
    fr.close()

    seed(0)
    cr = [0] * trucks       # Массив с маршрутами
    ratio = float(sum(var.demands)/(trucks*var.capacity))
    print()
    print("Name:", var.name, " Trucks:", trucks, " Cap:", var.capacity, " var.dimension:",
          var.dimension, " Goal:", goal, " Sum:", sum(var.demands),
          " Ratio:", ratio, sep='')
    start_time = datetime.now()
    for i in range(trucks):
        cr[i] = [0] * var.dimension

    # В случае неуспеха повторяем попытку с другим начальным распределением
    distr = False
    while distr == False:
        for r in range(trucks): # Обнуляем массив
            for i in range(var.dimension):
                cr[r][i] = 0
        # Случайное распределение клиентов по маршрутам с учетом вместимости
        for i in range(1,var.dimension):
            ok = False # i-тый клиент получил свой маршртут
            r = -1 # Признак того, что маршрут ещё не был присвоен
            '''
            Возможна ситуация, когда очередного клиента невозможно поместить в очередной маршрут.
            В этом случае нужно начать распределение заново.
            '''
            while ok == False:
                if r == -1:
                    r = randrange(trucks)
                    fail = 0
                else:
                    r += 1
                    r = r % trucks
                j = last_client(cr[r])
                cr[r][j] = i
                if calc_cap(cr[r],var.demands) > var.capacity: # Проверяем, помещается ли клиент в маршрут
                    cr[r][j] = 0
                    fail += 1
                    if fail > trucks:
                        i = 999
                        break
                else:
                    ok = True
        if i != 999:
            distr = True

    T = total_length( cr, var ) # Начальная "температура"
    print("T =",T)
    for count in range(1,MAX_COUNT):
        # Меняем 1 или 2 маршрута, определяется рандомно
        cswap(cr,var,0.002*(MAX_COUNT-count)/MAX_COUNT*goal,randrange(1,3))
        len2 = total_length( cr, var )
        if len2 <= goal:
            break;
    end_time = datetime.now()
    show_routes(cr,goal,len2)
    print( "Calculation time:", end_time - start_time )


Name:A-n32-k5 Trucks:5 Cap:100 var.dimension:32 Goal:784 Sum:410 Ratio:0.82
T = 1863
Route #1: 14 24 25 
Route #2: 1 12 30 16 
Route #3: 5 10 15 4 2 3 23 
Route #4: 26 7 13 19 17 31 21 
Route #5: 8 11 28 6 27 20 29 22 9 18 
Cost 775
Calculation time: 0:00:00.065859

Name:A-n33-k5 Trucks:5 Cap:100 var.dimension:33 Goal:661 Sum:446 Ratio:0.892
T = 1828
Route #1: 16 3 29 31 19 11 24 
Route #2: 2 12 10 17 9 
Route #3: 32 8 7 26 13 6 14 21 1 
Route #4: 28 18 23 22 15 
Route #5: 5 30 25 27 4 20 
Cost 658
Calculation time: 0:00:00.250334

Name:A-n33-k6 Trucks:6 Cap:100 var.dimension:33 Goal:742 Sum:541 Ratio:0.9016666666666666
T = 1334
Route #1: 28 25 27 30 10 
Route #2: 21 12 16 31 
Route #3: 13 5 17 6 7 9 15 
Route #4: 1 4 26 8 19 
Route #5: 18 20 2 3 22 23 24 
Route #6: 32 14 11 29 
Cost 738
Calculation time: 0:00:00.032911

Name:A-n34-k5 Trucks:5 Cap:100 var.dimension:34 Goal:778 Sum:460 Ratio:0.92
T = 1736
Route #1: 10 13 28 21 16 22 9 
Route #2: 5 24 1 27 7 6 15 
Route #3: 20 4 26 30 1

Route #1: 51 62 28 23 57 12 13 
Route #2: 7 48 55 41 2 16 50 
Route #3: 11 54 19 44 52 24 21 45 5 
Route #4: 43 15 4 3 36 35 37 
Route #5: 33 1 10 25 29 49 27 
Route #6: 17 39 46 64 6 31 26 
Route #7: 18 8 59 40 58 61 42 38 
Route #8: 30 32 20 56 53 
Route #9: 47 34 9 22 14 
Cost 1165
Calculation time: 0:00:01.326076

Name:A-n69-k9 Trucks:9 Cap:100 var.dimension:69 Goal:1159 Sum:845 Ratio:0.9388888888888889
T = 3546
Route #1: 43 62 56 36 1 48 
Route #2: 28 4 60 21 61 64 2 
Route #3: 23 44 15 3 6 30 18 31 
Route #4: 58 22 12 29 34 65 27 7 
Route #5: 39 40 38 35 45 13 
Route #6: 19 47 55 46 32 10 17 
Route #7: 24 66 68 51 9 53 33 49 
Route #8: 11 63 5 37 57 42 25 14 26 54 
Route #9: 52 50 16 8 20 41 59 
Cost 1159
Calculation time: 0:00:03.484650

Name:A-n80-k10 Trucks:10 Cap:100 var.dimension:80 Goal:1763 Sum:942 Ratio:0.942
T = 4602
Route #1: 38 76 45 22 4 
Route #2: 5 29 59 17 72 50 32 
Route #3: 49 40 62 24 68 16 43 46 25 64 
Route #4: 21 10 8 20 75 19 57 52 48 
Route #5: 36 1 77 74 3

Route #1: 33 57 37 8 41 39 55 23 
Route #2: 3 11 27 7 35 64 15 
Route #3: 50 19 40 53 38 25 60 4 
Route #4: 46 30 21 20 52 49 22 
Route #5: 31 13 5 48 9 54 14 62 
Route #6: 44 16 34 29 42 36 67 
Route #7: 47 43 66 65 12 17 
Route #8: 1 56 63 32 6 2 51 61 26 59 45 
Route #9: 58 28 24 10 18 
Cost 1247
Calculation time: 0:00:01.045710

Name:B-n78-k10 Trucks:10 Cap:100 var.dimension:78 Goal:1221 Sum:937 Ratio:0.937
T = 4009
Route #1: 47 49 63 62 37 3 
Route #2: 66 6 33 55 59 50 61 
Route #3: 40 18 16 68 74 28 19 4 36 
Route #4: 56 12 45 46 34 8 22 
Route #5: 17 64 11 26 44 23 77 1 67 
Route #6: 42 60 32 73 31 76 7 71 15 
Route #7: 27 5 54 20 13 35 
Route #8: 2 38 43 21 24 52 69 72 58 
Route #9: 65 51 57 29 70 75 30 
Route #10: 39 9 48 14 41 10 25 53 
Cost 1220
Calculation time: 0:00:01.804332

Name:E-n101-k14 Trucks:14 Cap:112 var.dimension:101 Goal:1071 Sum:1458 Ratio:0.9298469387755102
T = 3052
Route #1: 89 13 97 87 72 24 80 
Route #2: 12 77 68 81 9 
Route #3: 69 27 7 48 18 5 84 17 
Rout